# Artificial Intelligence
## L.EIC – 3rd Year/2nd Semester
### Exercise Sheet 1
# Solving Problems by Searching

## The Two Buckets Problem

<img src="https://qph.cf2.quoracdn.net/main-qimg-45726b16b460cae0147ae8ca245a8fb0-pjlq" width="250px" height="250px" align="right">

Two buckets of capacities **c1** (e.g. 4 liters) and **c2** (e.g. 3 liters), respectively, are initially empty. The buckets do not have any intermediate markings. The only operations you can perform are:

- Fill (completely) a bucket 
- Empty a bucket.
- Pour one bucket into the other (until the target one is full or the source one is empty).

The aim is to determine which operations to carry out so that the first bucket contains exactly **n** liters (e.g. 2 litres).

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

### States
Two variables representing the current amount of liters in a bucket, lets say **b1** and **b2** with ranges [0, c1] and [0, c2] respectively.


### Initial state
Any valid state.


### Operators
- Empty the first bucket `empty1`.
- Empty the second bucket `empty2`.

   Emptying an empty bucket does nothing

- Fill the first bucket `fill1`.
- Fill the second bucket `fill2`.

   Filling a full bucket does nothing

- Pour the maximum amount of liters from the first bucket into the second bucket `pour12`.
- Pour the maximum amount of liters from the second bucket into the first bucket `pour21`.

   The bucket we are pouring from needs to have a nonzero amount of liters and the bucket we are pouring to mustn't be already completely filled.
   
   Furthermore the amount poured is either the total liters of the bucket we are pouring from or the remaining liters until the bucket we are pouring into reaches its maximum capacity, whichever comes first.

### Cost
All operations have the same unit cost

What is the size of the state space for this problem? Represent the state space by drawing the possible objective states and displaying some of the possible transitions from the initial state.

Solve the problem by hand, using tree search. What solutions have you found?

### Buildind a computational approach to handle the problem

To build a program to solve the buckets problem, we will implement a solution that separates the problem definition from the algorithms used to traverse the state space. This way, we can reuse our implementations of the search strategies in other problems.

#### Representing the two buckets problem as a search problem

Let's start by defining a state for the buckets problem. For that, it'll suffice to aggregate two quantities, each representing the amount of water in one of the buckets. We also define a way of printing the state.

In [2]:
class BucketState:
    c1 = 4   # capacity for bucket 1
    c2 = 3   # capacity for bucket 2
    
    def __init__(self, b1, b2):
        self.b1 = b1
        self.b2 = b2

    '''needed for the visited list'''
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False

    def __ne__(self, other):
        """Overrides the default implementation (unnecessary in Python 3)"""
        return not self.__eq__(other)
    
    def __hash__(self):
        return hash((self.b1, self.b2)) 
    ''' - '''

    def __str__(self):
        return "(" + str(self.b1) + ", " + str(self.b2) + ")"

    def __repr__(self):
        return str(self)

Now we define each of the operators on states:

In [3]:
# emptying the first bucket
def empty1(state):
    if state.b1 > 0:
        return BucketState(0, state.b2)
    return None

# emptying the second bucket
def empty2(state):
    if state.b2 > 0:
        return BucketState(state.b1, 0)
    return None

def fill1(state):
    if state.b1 < state.c1:
        return BucketState(state.c1, state.b2)
    return None

def fill2(state):
    if state.b2 < state.c2:
        return BucketState(state.b1, state.c2)
    return None

def pour12(state):
    if state.b1 > 0 and state.b2 < state.c2:
        return BucketState(max(0, state.b1 - (state.c2 - state.b2)), min(state.c2, state.b2 + state.b1))
    return None

def pour21(state):
    if state.b2 > 0 and state.b1 < state.c1:
        return BucketState(min(state.c1, state.b1 + state.b2),max(0, state.b2 - (state.c1 - state.b1)))
    return None

The following function will aggregate all states that can be generated from a given one:

In [4]:
def child_bucket_states(state):
    new_states = []
    if(empty1(state)):
        new_states.append(empty1(state))
    if(empty2(state)):
        new_states.append(empty2(state))
    if(fill1(state)):
        new_states.append(fill1(state))
    if(fill2(state)):
        new_states.append(fill2(state))
    if(pour12(state)):
        new_states.append(pour12(state))
    if(pour21(state)):
        new_states.append(pour21(state))
    # if(pour12_fill2(state)):
    #     new_states.append(pour12_fill2(state))
    # if(pour12_empty1(state)):
    #     new_states.append(pour12_empty1(state))
    # if(pour21_fill1(state)):
    #     new_states.append(pour21_fill1(state))
    # if(pour21_empty2(state)):
    #     new_states.append(pour21_empty2(state))
    return new_states

Play around with the state transition operators and check if they are working properly:

In [5]:
s = BucketState(0, 0)
s = fill1(s)
print(s)

level1 = child_bucket_states(BucketState(0, 0))
print(level1)

child_bucket_states(level1[0])

# your code here


(4, 0)
[(4, 0), (0, 3)]


[(0, 0), (4, 3), (1, 3)]

Finally, we need to define the goal condition:

In [6]:
def goal_bucket_state(state):
    return state.b1 == 2

Test your goal condition:

In [7]:
print(goal_bucket_state(BucketState(0,0))) # false
print(goal_bucket_state(BucketState(1,1))) # false
print(goal_bucket_state(BucketState(2,0))) # true
print(goal_bucket_state(BucketState(2,3))) # true

False
False
True
True


#### Implementing search algorithms

Let us start by defining an appropriate structure to represent a node in a search tree. Each tree node will include:
- a state of the problem
- a link to its parent (to allow traveling from a leaf node towards the root of the tree)
- a list of child nodes

In [8]:
# A generic definition of a tree node holding a state of the problem
class TreeNode:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        if self.parent is None:
            self.depth = 0
        else:
            self.depth = self.parent.depth + 1

    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self

##### Breadth-first search

Based on this structure, we can now implement breadth-first search. Note that we want the implementation to be independent of the problem at hand (in this case, the two buckets problem).

In [9]:
from collections import deque

def breadth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state)   # create the root node in the search tree
    queue = deque([root])   # initialize the queue to store the nodes
    
    while queue:
        node = queue.popleft()   # get first element in the queue
        if goal_state_func(node.state):   # check goal state
            return node
        
        for state in operators_func(node.state):   # go through next states
            # create tree node with the new state
            child = TreeNode(state)
            
            # link child node to its parent in the tree
            node.add_child(child)
            
            # enqueue the child node
            queue.append(child)

    return None

We can now use this function to actually perform a breadth-first search on the buckets problem: we pass it the initial state, our goal condition function, and the function for obtaining child states.

In [10]:
goal = breadth_first_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states)
print(goal.state)

(2, 3)


In order to print the actual steps from the initial state to the last, we can take advantage of each node's link to its parent.

In [11]:
def print_solution(node):
    print(node.state, end='')
    if node.parent != None:
        print(' <- ', end='')
        print_solution(node.parent)
        return
    print()

Now we can print the solution:

In [12]:
print_solution(goal)

(2, 3) <- (4, 1) <- (0, 1) <- (1, 0) <- (1, 3) <- (4, 0) <- (0, 0)


If we need a description for each of the employed operators, we could have each operation function return also such a description, and modify the TreeNode class so that each node also includes a description of the edge to get there. We leave that as an exercise after class.

##### Depth-first search

Implement depth-first search (again, in a manner that is independent of the problem at hand). You can start from your breadth-first search implementation and with minor changes get an implementation for depth-first search.

In [13]:
def depth_first_search(initial_state, goal_state_func, operators_func):
    root = TreeNode(initial_state)   # create the root node in the search tree
    queue = deque([root])   # initialize the queue to store the nodes
    visited = set([])
    
    while queue:
        node = queue.popleft()   # get first element in the queue
        if goal_state_func(node.state):   # check goal state
            return node
        
        visited.add(node.state)
        
        for state in operators_func(node.state):   # go through next states
            if state in visited:
                continue
            # create tree node with the new state
            child = TreeNode(state)
            
            # link child node to its parent in the tree
            node.add_child(child)
            
            # enqueue the child node
            queue.appendleft(child)

    return None
    

Test it on the two buckets problem.

In [14]:
goal = depth_first_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states)
print(goal.state)
print_solution(goal)

(2, 3)
(2, 3) <- (4, 1) <- (0, 1) <- (1, 0) <- (1, 3) <- (4, 0) <- (4, 3) <- (4, 2) <- (3, 3) <- (3, 0) <- (0, 3) <- (0, 0)


If you are unable to get a solution, think about it: depth-first search is not a complete search method, and one of the reasons for that is if the state space contains cycles. As such, you need to make sure you avoid entering into a cycle by keeping a visited nodes list or set and checking that list whenever you generate a new state.

##### Depth-limited search

Another way to make it work is to impose a depth limit to the problem. Implement depth-limited search.

In [15]:
def depth_limited_search(initial_state, goal_state_func, operators_func, depth_limit):
    root = TreeNode(initial_state)
    queue = deque([root])
    
    while queue:
        node = queue.popleft()
        if goal_state_func(node.state):
            return node
        
        if node.depth > depth_limit:
            continue
        
        for state in operators_func(node.state):
            child = TreeNode(state, node)
            
            #node.add_child(child)
            queue.appendleft(child)

    return None


Test it on the two buckets problem.

In [16]:
goal = depth_limited_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states, 5)
print(goal.state)
print_solution(goal)


(2, 0)
(2, 0) <- (0, 2) <- (4, 2) <- (3, 3) <- (3, 0) <- (0, 3) <- (0, 0)


##### Iterative deepening search

Based on depth-limited, you can easily implement iterative-deepening search.

In [17]:
def iterative_deepening_search(initial_state, goal_state_func, operators_func, depth_limit=None):
    i = depth_limit or 0
    while True:
        result = depth_limited_search(initial_state, goal_state_func, operators_func, i)
        if result != None:
            return result
        if depth_limit != None and i >= depth_limit:
            return None
        i += 1

Again, test it on the two buckets problem.

In [18]:
goal = iterative_deepening_search(BucketState(0,0), 
                            goal_bucket_state, 
                            child_bucket_states,5)
print(goal.state)
print_solution(goal)

(2, 0)
(2, 0) <- (0, 2) <- (4, 2) <- (3, 3) <- (3, 0) <- (0, 3) <- (0, 0)


##### Greedy search

Start by defining an heuristic function and implement the Greedy Search algorithm (independent of the problem at hand)

In [19]:
def heuristic_bucket(node):
    return 2 - node.state.b1    

In [20]:
# check if state is already in the tree (from node to root)
def duplicate(state, node):
    while node != None:
        if node.state == state:
            return True
        node = node.parent
    return False

def greedy_search(initial_state, goal_state_func, operators_func, heuristic_func):
    root = TreeNode(initial_state)
    queue = deque([root])
    
    while queue:
        node = queue.popleft()
        if goal_state_func(node.state):
            return node
        
        for state in operators_func(node.state):
            if duplicate(state, node):
                continue

            child = TreeNode(state, node)
            child.heuristic = heuristic_func(child)
            queue.append(child)
            
        queue = deque(sorted(queue, key=lambda node: node.heuristic))
        
    return None
    

Test it on the two buckets problem.

In [21]:
goal = greedy_search(BucketState(0,0), goal_bucket_state, child_bucket_states, heuristic_bucket)
print(goal.state)
print_solution(goal)

(2, 3)
(2, 3) <- (4, 1) <- (0, 1) <- (1, 0) <- (1, 3) <- (4, 0) <- (0, 0)


##### A* Algorithm

Reuse the heuristic function defined before. This is very similar to greedy search, the difference is that it takes into account the cost of the path so far

In [22]:
def a_star_search(initial_state, goal_state_func, operators_func, heuristic):
    root = TreeNode(initial_state)
    queue = [root]
    
    while queue:
        node = queue.pop(0)
        if goal_state_func(node.state):
            return node
        
        for state in operators_func(node.state):
            if duplicate(state, node):
                continue

            child = TreeNode(state, node)
            child.heuristic = heuristic(child)
            # child.cost = node.cost + 1
            child.f = child.heuristic + node.depth
            queue.append(child)
        
        queue.sort(key=lambda node: node.f)
        
    return None

Again, test it on the two buckets problem.

In [23]:
goal = a_star_search(BucketState(0,0), goal_bucket_state, child_bucket_states, heuristic_bucket)
print(goal.state)
print_solution(goal)

(2, 3)
(2, 3) <- (4, 1) <- (0, 1) <- (1, 0) <- (1, 3) <- (4, 0) <- (0, 0)


## The Missionaries and Cannibals Problem 

<img src="https://www.gamezkingdom.com/content/images/thumbs/0002926_missionaries-and-cannibals.jpeg" width="250px" height="250px" align="right">

Three missionaries and three cannibals are on one of the banks of the river with a boat that only takes one or two people. The boat cannot travel the river alone.

The goal is to find a way to get the six to the other bank of the river without ever leaving more cannibals than missionaries on one of the banks (even at the instant they leave/join the boat) during the process.

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

Solve the problem by hand, using tree search. What solutions have you found?

Represent the problem as a search problem and take advantage of the implemented search algorithms to find solutions!

In [24]:
class MissionariesCannibalsState:
    def __init__(self, m_left, c_left, m_right, c_right, boat):
        self.m_left = m_left
        self.c_left = c_left
        self.m_right = m_right
        self.c_right = c_right
        self.boat = boat
        
    def __eq__(self, other):
        return self.m_left == other.m_left and self.c_left == other.c_left and self.boat == other.boat
    
    def __hash__(self):
        return hash((self.m_left, self.c_left, self.boat))
    
    def __str__(self):
        # draw the missionaries as M and cannibals as C with | | representing the sides of the river
        ml = 'M' * self.m_left
        cl = 'C' * self.c_left
        mr = 'M' * self.m_right
        cr = 'C' * self.c_right

        return '({} {}|{}|{} {})'.format(ml, cl, 'B  ' if self.boat == 'left' else '  B', mr, cr)

        #return 'Left: M:{} C:{} | Right: M:{} C:{} | Boat: {}'.format(self.m_left, self.c_left, self.m_right, self.c_right, self.boat)
    
    def __repr__(self):
        return self.__str__()

    def is_valid(self):
        if self.m_left < 0 or self.c_left < 0 or self.m_right < 0 or self.c_right < 0:
            return False

        if self.m_left < self.c_left and self.m_left > 0:
            return False

        if self.m_right < self.c_right and self.m_right > 0:
            return False

        return True


def travel(state, missionaries, cannibals):
    # check if the number of missionaries and cannibals is valid
    if missionaries < 0 or cannibals < 0:
        #print('Invalid number of missionaries({}) or cannibals({})'.format(missionaries, cannibals))
        return None

    # must have at least one missionary or cannibal
    if missionaries == 0 and cannibals == 0:
        #print('Must have at least one missionary or cannibal')
        return None

    # boat has 2 seats
    if missionaries + cannibals > 2:
        #print('Boat has only 2 seats: {} missionaries and {} cannibals were given'.format(missionaries, cannibals))
        return None

    if state.boat == 'left':
        # if not check_left(state, missionaries, cannibals):
        #     return None
        res = MissionariesCannibalsState(state.m_left - missionaries, 
            state.c_left - cannibals, state.m_right + missionaries, state.c_right + cannibals, 'right')
    else:
        # if not check_right(state, missionaries, cannibals):
        #     return None
        res = MissionariesCannibalsState(state.m_left + missionaries,
            state.c_left + cannibals, state.m_right - missionaries, state.c_right - cannibals, 'left')
    
    if res.is_valid():
        return res
    else:
        return None

def goal_missionaries_cannibals(state):
    return state.m_left == 0 and state.c_left == 0 and state.m_right == 3 and state.c_right == 3

# test travel function
def test_travel(state):
    print('0: {}'.format(travel(state, 1, 2))) # error
    print('1: {}'.format(travel(state, 2, 0))) # error

    s2 = travel(state, 1, 1) # move 1 missionary and 1 cannibal to the right
    print('2: {}'.format(s2))

    s3 = travel(s2, 1, 0) # move 1 missionary to the left
    print('3: {}'.format(s3))

    s4 = travel(s3, 2, 0) # should be None
    print('4: {}'.format(s4))

#test_travel(MissionariesCannibalsState(3, 3, 0, 0, 'left'))

def child_missionaries_cannibals(state):
    children = []
    for m in range(3):
        for c in range(3):
            child = travel(state, m, c)
            if child:
                children.append(child)
    return children

# test child_missionaries_cannibals function
def test_child_missionaries_cannibals(state):
    print('Children of {}:'.format(state))
    for child in child_missionaries_cannibals(state):
        print(child)

# test_child_missionaries_cannibals(MissionariesCannibalsState(3, 3, 0, 0, 'left'))
# print()
# test_child_missionaries_cannibals(MissionariesCannibalsState(2, 2, 1, 1, 'right'))

# BFS
print('BFS')
goal = breadth_first_search(MissionariesCannibalsState(3, 3, 0, 0, 'left'), goal_missionaries_cannibals, child_missionaries_cannibals)
#print(goal.state)
print_solution(goal)
print()

# DFS
print('DFS')
goal = depth_first_search(MissionariesCannibalsState(3, 3, 0, 0, 'left'), goal_missionaries_cannibals, child_missionaries_cannibals)
#print(goal.state)
print_solution(goal)


BFS
( |  B|MMM CCC) <- ( CC|B  |MMM C) <- ( C|  B|MMM CC) <- ( CCC|B  |MMM ) <- ( CC|  B|MMM C) <- (MM CC|B  |M C) <- (M C|  B|MM CC) <- (MMM C|B  | CC) <- (MMM |  B| CCC) <- (MMM CC|B  | C) <- (MMM C|  B| CC) <- (MMM CCC|B  | )

DFS
( |  B|MMM CCC) <- (M C|B  |MM CC) <- ( C|  B|MMM CC) <- ( CCC|B  |MMM ) <- ( CC|  B|MMM C) <- (MM CC|B  |M C) <- (M C|  B|MM CC) <- (MMM C|B  | CC) <- (MMM |  B| CCC) <- (MMM CC|B  | C) <- (MM CC|  B|M C) <- (MMM CCC|B  | )


## N-Puzzle Problem

The objective of this exercise is the application of search methods, with emphasis on informed
search methods and the A\* algorithm, to solve the well-known N-Puzzle problem. The desired
objective self for the puzzle is as follows (0 represents the empty space):

<table>
<tr><th>9Puzzle</th><th>16Puzzle</th></tr>
<tr>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 2   | 3   |
| 4   | 5   | 6   |
| 7   | 8   | 0   |


</td>
<td>

|     |     |     |     |
| --- | --- | --- | --- |
| 1   | 2   | 3   | 4   |
| 5   | 6   | 7   | 8   |
| 9   | 10  | 11  | 12  |
| 13  | 14  | 15  | 0   |

</td>
</tr>
</table>

Starting from a given initial state, the goal is to determine which operations to perform to
solve the puzzle, reaching the desired objective self.

Formulate this problem as a search problem by defining the state representation, initial state, operators (their name, preconditions, effects, and cost), and objective test.

Represent the problem as a search problem and take advantage of the implemented search algorithms to find solutions!

For the Greedy Search and the A* Algorithm suppose the following heuristics for these methods:
- H1 - Number of incorrect placed pieces;
- H2 - Sum of manhattan distances from incorrect placed pieces to their correct places. 

Finally Compare the results obtained concerning execution time and memory space occupied in solving the following problems using the previous methods

<table>
<tr><th>Prob. 1</th><th>Prob. 2</th><th>Prob. 3</th><th>Prob. 4</th></tr>
<tr>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 2   | 3   |
| 5   | 0   | 6   |
| 4   | 7   | 8   | 

</td>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 3   | 6   |
| 5   | 2   | 0   |
| 4   | 7   | 8   | 

</td>
<td>

|     |     |     |
| --- | --- | --- |
| 1   | 6   | 2   |
| 5   | 7   | 3   |
| 0   | 4   | 8   | 

</td>
<td>

|     |     |     |     |
| --- | --- | --- | --- |
| 5   | 1   | 3   | 4   |
| 2   | 0   | 7   | 8   |
| 10  | 6   | 11  | 12  |
| 9   | 13  | 14  | 15  |

</td>
</tr>
</table>

In [ ]:
# the following arrays represent the problem for testing
initial_states = [
    [[1, 2, 3], [5, 0, 6], [4, 7, 8]],
    [[1, 3, 6], [5, 2, 0], [4, 7, 8]],
    [[1, 6, 2], [5, 7, 3], [0, 4, 8]],
    [[5, 1, 3, 4], [2, 0, 7, 8], [10, 6, 11, 12], [9, 13, 14, 15]]
]

In [ ]:
# your code here
